In [1]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer, TFAutoModelForSequenceClassification
from datasets import load_dataset, DatasetDict
import shutil

In [2]:
label_to_id = {
    "restaurant_reviews": 0,
    "nutrition_info": 1,
    "account_blocked": 2,
    "oil_change_how": 3,
    "time": 4,
    "weather": 5,
    "redeem_rewards": 6,
    "interest_rate": 7,
    "gas_type": 8,
    "accept_reservations": 9,
    "smart_home": 10,
    "user_name": 11,
    "report_lost_card": 12,
    "repeat": 13,
    "whisper_mode": 14,
    "what_are_your_hobbies": 15,
    "order": 16,
    "jump_start": 17,
    "schedule_meeting": 18,
    "meeting_schedule": 19,
    "freeze_account": 20,
    "what_song": 21,
    "meaning_of_life": 22,
    "restaurant_reservation": 23,
    "traffic": 24,
    "make_call": 25,
    "text": 26,
    "bill_balance": 27,
    "improve_credit_score": 28,
    "change_language": 29,
    "no": 30,
    "measurement_conversion": 31,
    "timer": 32,
    "flip_coin": 33,
    "do_you_have_pets": 34,
    "balance": 35,
    "tell_joke": 36,
    "last_maintenance": 37,
    "exchange_rate": 38,
    "uber": 39,
    "car_rental": 40,
    "credit_limit": 41,
    "change_volume": 42,
    "shopping_list": 43,
    "expiration_date": 44,
    "routing": 45,
    "meal_suggestion": 46,
    "tire_change": 47,
    "todo_list": 48,
    "card_declined": 49,
    "rewards_balance": 50,
    "change_accent": 51,
    "vaccines": 52,
    "reminder_update": 53,
    "food_last": 54,
    "change_ai_name": 55,
    "bill_due": 56,
    "who_do_you_work_for": 57,
    "share_location": 58,
    "international_visa": 59,
    "calendar": 60,
    "translate": 61,
    "carry_on": 62,
    "book_flight": 63,
    "insurance_change": 64,
    "todo_list_update": 65,
    "timezone": 66,
    "cancel_reservation": 67,
    "transactions": 68,
    "credit_score": 69,
    "report_fraud": 70,
    "spending_history": 71,
    "directions": 72,
    "spelling": 73,
    "insurance": 74,
    "what_is_your_name": 75,
    "reminder": 76,
    "where_are_you_from": 77,
    "distance": 78,
    "payday": 79,
    "flight_status": 80,
    "find_phone": 81,
    "greeting": 82,
    "alarm": 83,
    "order_status": 84,
    "confirm_reservation": 85,
    "cook_time": 86,
    "damaged_card": 87,
    "reset_settings": 88,
    "pin_change": 89,
    "replacement_card_duration": 90,
    "new_card": 91,
    "roll_dice": 92,
    "income": 93,
    "taxes": 94,
    "date": 95,
    "who_made_you": 96,
    "pto_request": 97,
    "tire_pressure": 98,
    "how_old_are_you": 99,
    "rollover_401k": 100,
    "pto_request_status": 101,
    "how_busy": 102,
    "application_status": 103,
    "recipe": 104,
    "calendar_update": 105,
    "play_music": 106,
    "yes": 107,
    "direct_deposit": 108,
    "credit_limit_change": 109,
    "gas": 110,
    "pay_bill": 111,
    "ingredients_list": 112,
    "lost_luggage": 113,
    "goodbye": 114,
    "what_can_i_ask_you": 115,
    "book_hotel": 116,
    "are_you_a_bot": 117,
    "next_song": 118,
    "change_speed": 119,
    "plug_type": 120,
    "maybe": 121,
    "w2": 122,
    "oil_change_when": 123,
    "thank_you": 124,
    "shopping_list_update": 125,
    "pto_balance": 126,
    "order_checks": 127,
    "travel_alert": 128,
    "fun_fact": 129,
    "sync_device": 130,
    "schedule_maintenance": 131,
    "apr": 132,
    "transfer": 133,
    "ingredient_substitution": 134,
    "calories": 135,
    "current_location": 136,
    "international_fees": 137,
    "calculator": 138,
    "definition": 139,
    "next_holiday": 140,
    "update_playlist": 141,
    "mpg": 142,
    "min_payment": 143,
    "change_user_name": 144,
    "restaurant_suggestion": 145,
    "travel_notification": 146,
    "cancel": 147,
    "pto_used": 148,
    "travel_suggestion": 149,
}


In [3]:
dataset = load_dataset("clinc/clinc_oos", "plus")
def filter_intent(example):
    return example['intent'] != 42

# Apply the filter function to each split to remove OOS queries
dataset = DatasetDict({
    split: dataset[split].filter(filter_intent)
    for split in dataset
})
# Now we need to update the labels so that our model does not give an error. 
def update_intent(example):
    if example['intent'] == 150:
        example['intent'] = 42
    return example

dataset = DatasetDict({
    split: dataset[split].map(update_intent)
    for split in dataset
})
dataset

README.md:   0%|          | 0.00/24.0k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/312k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/136k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15250 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/5500 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'intent'],
        num_rows: 15000
    })
    validation: Dataset({
        features: ['text', 'intent'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['text', 'intent'],
        num_rows: 4500
    })
})

In [4]:
import json
from datasets import Dataset
# Load the dataset
file_path = "/kaggle/input/augmented-dataset-new/augmented_dataset_new.json"

filtered_data = []

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        record = json.loads(line.strip())
        intent_id = label_to_id[record["intent"]]
        # Extract only 'query' and 'intent'
        filtered_data.append({"query": record["query"], "intent": intent_id})

# Convert to Hugging Face Dataset
filtered_data = Dataset.from_list(filtered_data)

# Inspect the Dataset
print(len(filtered_data))




28860


In [5]:
filtered_data

Dataset({
    features: ['query', 'intent'],
    num_rows: 28860
})

In [6]:
filtered_data = filtered_data.rename_column('query', 'text')
filtered_data = filtered_data.rename_column('intent', 'label')


In [7]:
filtered_data

Dataset({
    features: ['text', 'label'],
    num_rows: 28860
})

In [8]:
dataset['train'] = filtered_data
dataset['validation'] = dataset['validation'].rename_column('intent', 'label')
dataset['test'] = dataset['test'].rename_column('intent', 'label')

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 28860
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4500
    })
})

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 28860
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4500
    })
})

In [11]:
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)
dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/28860 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

In [13]:
keep_columns = ["input_ids", "attention_mask", "label"]
dataset.set_format("numpy", columns=keep_columns)

In [14]:
BATCH_SIZE = 16

def to_tf_dataset(hf_dataset, shuffle=False):
    """Convert a Hugging Face Dataset split into a tf.data.Dataset (features, label)."""
    input_ids = hf_dataset["input_ids"]
    attention_mask = hf_dataset["attention_mask"]
    labels = hf_dataset["label"]

    # Commented out: "token_type_ids" (was used for BERT, not needed for RoBERTa),
    # RoBERTa typically doesn't use token_type_ids, so it's often absent.
    # token_type_ids = hf_dataset["token_type_ids"]

    ds = tf.data.Dataset.from_tensor_slices((
        {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            # "token_type_ids": token_type_ids,  # roberta does not need
        },
        labels
    ))

    if shuffle:
        ds = ds.shuffle(10000)
    ds = ds.batch(BATCH_SIZE)
    return ds

train_tf_dataset = to_tf_dataset(dataset["train"], shuffle=True)
val_tf_dataset   = to_tf_dataset(dataset["validation"], shuffle=False)
test_tf_dataset  = to_tf_dataset(dataset["test"], shuffle=False)


In [15]:
num_labels=150
lr=1e-5
num_epochs=40
model = TFAutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=num_labels
)
# AdamW from tensorflow
optimizer = tf.keras.optimizers.AdamW(learning_rate=lr, weight_decay=0.01)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [16]:
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
# Train
history = model.fit(
    train_tf_dataset,
    validation_data=val_tf_dataset,
    epochs=num_epochs
)

Epoch 1/40
1804/1804 [==============================] - 1096s 585ms/step - loss: 3.9117 - accuracy: 0.3694 - val_loss: 1.9363 - val_accuracy: 0.8100
Epoch 2/40
1804/1804 [==============================] - 1045s 579ms/step - loss: 2.4766 - accuracy: 0.5876 - val_loss: 0.6959 - val_accuracy: 0.9347
Epoch 3/40
1804/1804 [==============================] - 1042s 578ms/step - loss: 1.8908 - accuracy: 0.6346 - val_loss: 0.3622 - val_accuracy: 0.9520
Epoch 4/40
1804/1804 [==============================] - 1043s 578ms/step - loss: 1.6353 - accuracy: 0.6609 - val_loss: 0.2474 - val_accuracy: 0.9657
Epoch 5/40
1804/1804 [==============================] - 1041s 577ms/step - loss: 1.4753 - accuracy: 0.6814 - val_loss: 0.1990 - val_accuracy: 0.9683
Epoch 6/40
1804/1804 [==============================] - 1041s 577ms/step - loss: 1.3472 - accuracy: 0.7037 - val_loss: 0.1928 - val_accuracy: 0.9623
Epoch 7/40
1804/1804 [==============================] - 1043s 578ms/step - loss: 1.2351 - accuracy: 0.7217

In [ ]:
model.evaluate(test_tf_dataset)

In [ ]:
output_dir = "/kaggle/working/fine_tuned_roberta_40epoch"

# Save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")


In [ ]:
import shutil

shutil.make_archive("fine_tuned_roberta", 'zip', output_dir)
print("Zipped model saved as fine_tuned_roberta.zip")


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# Load the model
model = AutoModelForSequenceClassification.from_pretrained(output_dir, from_tf=True)
tokenizer = AutoTokenizer.from_pretrained(output_dir, from_tf=True)


In [ ]:
import torch
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
def evaluate_model_with_cm(model, test_dataset, num_classes=150):
    """
    Evaluate the model with corrected per-class accuracy calculation.
    """
    # Set model to evaluation mode
    model.eval()
    
    # Lists to store predictions and true labels
    all_predictions = []
    all_labels = []
    
    # Disable gradient calculations for inference
    with torch.no_grad():
        for batch in tqdm(test_dataset, desc="Evaluating"):
            inputs = batch[0]
            labels = batch[1]
            
            device = next(model.parameters()).device
            input_dict = {
                'input_ids': torch.tensor(inputs['input_ids'].numpy()).to(device),
                'attention_mask': torch.tensor(inputs['attention_mask'].numpy()).to(device)
            }
            
            outputs = model(**input_dict)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1).cpu().numpy()
            
            all_predictions.extend(predictions)
            all_labels.extend(labels.numpy())
    
    # Convert to numpy arrays
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    
    # Calculate general metrics
    metrics = {
        'accuracy': accuracy_score(all_labels, all_predictions),
        'f1_macro': f1_score(all_labels, all_predictions, average='macro'),
        'precision_macro': precision_score(all_labels, all_predictions, average='macro'),
        'recall_macro': recall_score(all_labels, all_predictions, average='macro')
    }
    
    # Calculate full confusion matrix
    full_cm = confusion_matrix(all_labels, all_predictions)
    metrics['full_confusion_matrix'] = full_cm
    
    # Calculate per-class metrics
    class_metrics = []
    for i in range(num_classes):
        # Get indices for this class
        class_mask = (all_labels == i)
        if np.any(class_mask):
            # Get predictions and true labels for this class
            class_preds = all_predictions[class_mask]
            class_true = all_labels[class_mask]
            
            # Calculate metrics for this class
            class_acc = accuracy_score(class_true, class_preds)
            
            # Calculate precision and recall for this class
            class_precision = precision_score(all_labels == i, all_predictions == i)
            class_recall = recall_score(all_labels == i, all_predictions == i)
            
            # Store all metrics
            class_metrics.append({
                'class_id': i,
                'accuracy': class_acc,
                'precision': class_precision,
                'recall': class_recall,
                'support': np.sum(class_mask)  # number of samples for this class
            })
    
    # Sort classes by accuracy
    sorted_classes = sorted(class_metrics, key=lambda x: x['accuracy'])
    
    # Get worst, middle, and best performing classes
    worst_class = sorted_classes[0]
    middle_class = sorted_classes[len(sorted_classes)//2]
    best_class = sorted_classes[-1]
    
    # Calculate confusion matrices for selected classes
    def get_binary_confusion_matrix(true_labels, pred_labels, target_class):
        """Convert multi-class labels to binary for one specific class"""
        true_binary = (true_labels == target_class).astype(int)
        pred_binary = (pred_labels == target_class).astype(int)
        return confusion_matrix(true_binary, pred_binary)
    
    selected_cms = {
        'worst_class': {
            'class_id': worst_class['class_id'],
            'accuracy': worst_class['accuracy'],
            'precision': worst_class['precision'],
            'recall': worst_class['recall'],
            'support': worst_class['support'],
            'confusion_matrix': get_binary_confusion_matrix(all_labels, all_predictions, worst_class['class_id'])
        },
        'middle_class': {
            'class_id': middle_class['class_id'],
            'accuracy': middle_class['accuracy'],
            'precision': middle_class['precision'],
            'recall': middle_class['recall'],
            'support': middle_class['support'],
            'confusion_matrix': get_binary_confusion_matrix(all_labels, all_predictions, middle_class['class_id'])
        },
        'best_class': {
            'class_id': best_class['class_id'],
            'accuracy': best_class['accuracy'],
            'precision': best_class['precision'],
            'recall': best_class['recall'],
            'support': best_class['support'],
            'confusion_matrix': get_binary_confusion_matrix(all_labels, all_predictions, best_class['class_id'])
        }
    }
    
    metrics['selected_confusion_matrices'] = selected_cms
    metrics['class_metrics'] = sorted_classes
    
    return metrics

def print_class_details(metrics, class_type):
    """Print detailed metrics for a specific class."""
    class_info = metrics['selected_confusion_matrices'][class_type]
    cm = class_info['confusion_matrix']
    
    print(f"\n{class_type.upper()} CLASS (ID: {class_info['class_id']}):")
    print(f"Accuracy: {class_info['accuracy']:.4f}")
    print(f"Precision: {class_info['precision']:.4f}")
    print(f"Recall: {class_info['recall']:.4f}")
    print(f"Number of samples: {class_info['support']}")
    print("\nConfusion Matrix:")
    print(f"TN: {cm[0,0]}, FP: {cm[0,1]}")
    print(f"FN: {cm[1,0]}, TP: {cm[1,1]}")

roberta_results = evaluate_model_with_cm(model, test_tf_dataset)